In [1]:
import torch
import torch.nn as nn
from torch.func import vmap, jacfwd
from dataclasses import dataclass

import numpy as np

from src.modules.rhot import Potts, make_potts
from src.modules.sampler import DiscreteJarzynskiIntegrator
from argparse import Namespace

In [2]:
device = torch.device('cuda')

In [3]:
config = Namespace(path='linear',L=6, n_cat=5,J_0=0, J_1=10, B_0=0, B_1=0, mu_0=0, mu_1=0, beta_0=1, beta_1=1)
potts_model = make_potts(config)

Js: tensor(10.) tensor(0.)
Bs: tensor(0.) tensor(0.)
betas: tensor(1.) tensor(1.)


Set hyperparameter

In [4]:
from src.nets.conv import ConvPottsRateMatrix
potts_matrix = ConvPottsRateMatrix(n_cat=config.n_cat,kernel_sizes=[3,5,7], num_channels=4).to(device)

In [5]:
eps = 0.05
ts = torch.linspace(0,1,101)
transport = True
n_save = 10
resample = False
resample_thres = 0.7
turn_off_jarz = False

In [6]:
# @dataclass
# class PottsJarzynskiIntegrator:
#     dpath: Potts
#     eps: torch.tensor
#     ts: torch.tensor
#     Qt_net: nn.Module = None,
#     transport: bool = False
#     n_save: int  = 10
#     resample: bool = False
#     resample_thres: float = 0.7
#     turn_off_jarz: bool = False
#     n_mcmcm_per_net: int = 1

#     def __post_init__(self) -> None:
#         """Initialize time grid and step size."""
#         self.n_step = len(self.ts) - 1
#         assert self.n_save > 0 ## will always save the first sample
#         assert self.n_step % self.n_save == 0, f"{self.n_step} is not divisible by {self.n_save}"
#         self.save_freq = self.n_step // self.n_save
        
        
#     def ess(self, At):
#         return torch.mean(torch.exp(At))**2 / torch.mean(torch.exp(2*At))
        
#     def _single_step_potts_glauber(self, cfg, t, dt):
#         """
#         Perform a Glauer update on Potts model for a single spin configuration (cfg).

#         cfg : (L, L)
#         t: float
#         dt: float

#         Returns
#         new_cfg:  (L, L)

        
#         """
#         raise NotImplementedError()
        
#         # J, mu, B, beta = self.dpath.setup_params(t)
    
#         # L = cfg.shape[0]
        
#         # row = np.random.randint(0, L, ())
#         # col = np.random.randint(0, L, ())
#         # spins = cfg[row, col]
    
#         # # neighbors
#         # up_row = (row - 1) % L
#         # down_row = (row + 1) % L
#         # left_col = (col - 1) % L
#         # right_col = (col + 1) % L

#         # # local neighbor interaction
#         # neighbor_sum = cfg[up_row, col] + cfg[down_row, col] + cfg[row, left_col] + cfg[row, right_col]
#         # local_energy = J * neighbor_sum + mu * B
#         # delta_E = 2.0 * spins * local_energy
    
#         # acc = torch.exp(-beta * delta_E)
#         # acc = torch.minimum(acc, torch.tensor(1)) 


#         # flip =  (torch.rand(1, device=cfg.device) < acc)
#         # # minus or plus 1 depending on if flip is true or flase respectively
#         # pm1 = torch.where(flip, torch.tensor(-1.0, device=cfg.device), torch.tensor(1.0, device=cfg.device))
#         # cfg[row, col] = pm1 * cfg[row, col] 

#         # return cfg

#     def metropolis_step(self, cfgs, t, dt):
#         """
#         Perform a Glauber update on a batch of spin configurations (cfgs).
#         """
#         bs = cfgs.shape[0]
#         return vmap(self._single_step_potts_glauber, in_dims = (0, 0, 0),
#                     out_dims=(0), randomness='different')(cfgs, t, dt)

#     def sigma_step(self, sigma, t, dt):
#         L = sigma.shape[-1]
#         if self.transport:
#             with torch.no_grad():
#                 sigma_new = self.Qt_net.sample_next_step(sigma, t, dt)
#         else:
#             sigma_new = sigma
#         for idx in range(n_mcmcm_per_net):
#             sigma_new = self.metropolis_step(sigma_new,t,dt)
#         return sigma_new, None
        
#     def A_step(self, sigma, t, A, dt):
#         L = sigma.shape[-1]
#         if self.transport and not self.turn_off_jarz:   
#             _, delta_H_neighbors = self.dpath.Ut(sigma, t)
#             lh_ratios = torch.exp(-delta_H_neighbors)
#             with torch.no_grad():
#                 jarz_corrector = self.Qt_net.get_jarzinsky_corrector(sigma, t, lh_ratios)
#             return A - dt * self.dpath.dtUt(sigma,t)[0] - dt * jarz_corrector
#         else:
#             return A - dt * self.dpath.dtUt(sigma,t)[0]
    
#     def rollout(self, sigma_init):
#         """
#         Performs the rollout
#         """
#         bs = sigma_init.shape[0]
#         sigma_stack = torch.zeros((self.n_save+1, *sigma_init.shape)).to(sigma_init)
#         As = torch.zeros((self.n_save+1, bs)).to(sigma_init)

#         # initialize weights A to zero
#         A = torch.zeros(bs).to(sigma_init)
#         sigma = sigma_init
        
#         sigma_stack[0] = sigma_init.detach()
#         As[0] = A
        
#         save_ind = 1
#         resample_count = 0
#         for i, t in enumerate(self.ts[:-1]):
#             dt = (self.ts[i+1] - self.ts[i]).repeat(len(sigma)).to(sigma_init.device) ### assumes first element in self.ts is 0
#             t = t.repeat(len(sigma)).to(sigma_init.device) #.requires_grad_(True)
#             sigma = sigma.to(sigma_init)
#             A = A.to(sigma_init)
#             sigma_new, _ = self.sigma_step(sigma, t, dt)
#             A_new = self.A_step(sigma, t, A, dt)

#             t_new = self.ts[i+1].repeat(len(sigma)) #.requires_grad_(True)

            
#             if self.ess(A_new.double()) < self.resample_thres and self.resample:
#                 resample_count +=1
#                 print("i:", i, "resample cout:", resample_count)
#                 weights = torch.softmax(A_new, dim = 0)
#                 indices = systematic_resample(weights)
#                 sigma_new   = torch.clone(sigma_new[indices])
#                 A_new   = torch.zeros(len(weights)).to(sigma_init)
                
#             if (i + 1) % self.save_freq == 0:            
#                 sigma_stack[save_ind] = torch.clone(sigma_new.detach())
#                 As[save_ind] = torch.clone(A_new.detach())
#                 save_ind += 1

#             sigma = sigma.detach()
#             del sigma
#             sigma = sigma_new.detach()
#             A = A_new.detach()
#             del sigma_new, A_new

#         sigma_stack[-1] = sigma.detach()
#         As[-1] = A.detach()

#         return sigma_stack, As

In [7]:
integrator = DiscreteJarzynskiIntegrator(dpath=potts_model,
    eps=eps,
    ts=ts,
    Qt_net=potts_matrix.to(device),
    transport=transport,
    n_save=n_save,
    resample=resample,
    resample_thres=resample_thres,
    model_class="potts",
    turn_off_jarz=turn_off_jarz, 
    n_mcmc_per_net=0)

In [8]:
for idx in range(integrator.n_mcmc_per_net):
    print("Hello")

In [9]:
bs = 10
x = torch.randint(0,config.n_cat,size=(bs,config.L,config.L)).to(device)
t = torch.rand(size=(bs,)).to(device)

In [10]:
integrator.n_mcmc_per_net

0

In [14]:
sigma_stack, As = integrator.rollout(x)

In [15]:
potts_model.dtUt(x,t)[0].shape

torch.Size([10])

In [16]:
sigma_stack.shape

torch.Size([11, 10, 6, 6])